In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_property2_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_property_two_year'].values

In [3]:
#### CART
depth = [1,2,3]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [60, 80, 100]
depth = [1]
learning_rate = [0.05]
holdout_split = [0.7, 0.9]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
cart_summary

{'best_auc': 0.6999708369305008,
 'auc_diff': 0.033286240822143376,
 'best_param': {'max_depth': 3,
  'min_impurity_decrease': 0.001,
  'min_samples_split': 2},
 'KY_score': 0.5226884321300713}

In [6]:
 ebm_summary

{'best_auc': 0.7356115487523278,
 'auc_diff': 0.053433603372518235,
 'best_param': {'holdout_split': 0.7,
  'learning_rate': 0.05,
  'max_tree_splits': 1,
  'n_estimators': 100},
 'KY_score': 0.6204796278640818}

### Lasso Stumps

In [7]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_property_two_year'].values

## columns 
cols = KY_X.columns

In [22]:
c_grid = {'C':[0.03, 0.05, 0.07, 0.09]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [23]:
stumps_summary

{'best_auc': 0.7106485290634161,
 'best_params': {'C': 0.09},
 'auc_diffs': 0.05678128120547987,
 'KY_score': 0.5781027839197072}

### RiskSLIM

In [24]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.09, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

35

In [25]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [26]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number = 10, 
                                max_runtime=200, 
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 01:41 PM | 1501 rows in lookup table
09/17/19 @ 01:41 PM | ------------------------------------------------------------
09/17/19 @ 01:41 PM | runnning initialization procedure
09/17/19 @ 01:41 PM | ------------------------------------------------------------
09/17/19 @ 01:41 PM | CPA produced 2 cuts
09/17/19 @ 01:41 PM | running naive rounding on 73 solutions
09/17/19 @ 01:41 PM | best objective value: 0.2762
09/17/19 @ 01:41 PM | rounding produced 5 integer solutions
09/17/19 @ 01:41 PM | best objective value is 0.2940
09/17/19 @ 01:41 PM | running sequential rounding on 73 solutions
09/17/19 @ 01:41 PM | best objective value: 0.2762
09/17/19 @ 01:41 PM | sequential rounding produced 6 integer solutions
09/17/19 @ 01:41 PM | best objective value: 0.2856
09/17/19 @ 01:41 PM | polishing 11 solutions
09/17/19 @ 01:41 PM | best objective value: 0.2856
09/17/19 @ 01:41 PM | polishing produced 5 integer solutions
09/17/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 01:41 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2845.
Tried aggregator 1 time.
Reduced MIP has 37 rows, 74 columns, and 143 nonzeros.
Reduced MIP has 35 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

In [27]:
riskslim_summary['KY_auc']

[0.5937782366348155]

### Corel

In [2]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_property_two_year'].values

In [3]:
KY_score, FL_score = Corel.corel_cv(KY_X, KY_Y, FL_X, FL_Y, max_card=2, c=1e-3, seed=816)

In [16]:
#### save results
summary_property2_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary, 
                                   "Corel": KY_score}
%store summary_property2_fl_inter_model

Stored 'summary_general2_fl_inter_model' (dict)


In [29]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Property",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_auc'][0], 3), 
            round(KY_score, 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)